# 実験
Case1・example_dataframe + 減衰なし
Case2・example_dataframe + 減衰あり
Case3・正規化_dataframe + 減衰なし
Case4・正規化_dataframe + 減衰あり

In [1]:
# sapnet.pyをインポート
from sapnet import sapnet
import random
import numpy as np
import datetime
import os


random.seed(314) # 乱数シードを314に設定

def console(attenuation_value):
    # 計算
    getdatas=[1,2,4,8,16,32,64,128]

    getdatas = []
    for i in range(1, 101):
        getdatas.append(i * 10)
        
    message=''

    for getdata in getdatas:
        # 10回の施行を見る
        line_data = []
        for j in range(1):
            # 100回の拡散をランダムで行う
            df = sapnet.example_dataframe_uniform()

            for i in range(getdata):
                df = sapnet.stimulus_calc(df=df, stimulus=random.randint(1, len(df)))
                df = sapnet.attenuation(df,attenuation_value)
                diagonal_matrix = np.diag(df.iloc[:, 1:].values)
            line_data.append(diagonal_matrix)

        # 行列の転置を求めることにより、それぞれの拡散の値を行列化
        # 行列化したそれぞれの要素を平均し、100回の拡散平均を求めます。
        transposed_line_data = np.transpose(line_data)

        # 各転置行列の平均を算出し、リストに追加
        averaged_lists = [np.mean(transposed_row) for transposed_row in transposed_line_data]

        # 最後に生成したリストの平均を求めて出力
        overall_average = np.mean(averaged_lists)
        add_message = ",".join(map(str, averaged_lists)) + ",{}".format(overall_average)
        message = message + '\n' + add_message

    return message

def makeup_folder():
    # 現在時刻を取得
    current_time = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    # 現在時刻を使用することでアウトプットファルダを重複なく作成することができる
    folder_name = f"./Output_{current_time}"
    # if not os.path.exists(folder_name):
    #     os.makedirs(folder_name)
    # 現在時刻を使用することでアウトプットファルダを重複なく作成することができる
    csv_folder_name = folder_name+"/CSV_source/"
    if not os.path.exists(csv_folder_name):
        os.makedirs(csv_folder_name)

    return csv_folder_name

def save_csv(text, filename):
    with open(filename, 'w') as file:
        file.write(text)

folder_name = makeup_folder()
for i in range(1, 101):
    attenuation_value = i / 100
    text = console(attenuation_value)
    formatted_number = str(i).zfill(3)
    filename = f"{folder_name}simulation_{formatted_number}.csv"
    save_csv(text, filename)

In [ ]:
import os
import pandas as pd

# フォルダのパスを設定
folder_path = folder_name

# 保存するExcelファイルの名前を設定（修正）
excel_file_name = f"{folder_path}combined_excel.xlsx"

# Excelライターを作成
writer = pd.ExcelWriter(excel_file_name, engine='openpyxl')

# フォルダ内の各ファイルに対して
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        # ファイルパスを作成
        file_path = os.path.join(folder_path, filename)
        
        # CSVファイルを読み込む
        df = pd.read_csv(file_path)
        
        # シート名をファイル名（拡張子なし）に設定
        sheet_name = os.path.splitext(filename)[0]
        
        # DataFrameをExcelシートに書き込む
        df.to_excel(writer, sheet_name=sheet_name, index=False,header=False)  # ここで index=True が使われていることに注意してください

# Excelファイルを保存（修正）
writer.close()
